We want the neural net to be given the same size volume at each episode.



First, we upsample the SWC files to make sure to get rid of huge inter-node distances, which can cause problems when trying to render a volume.

In [ ]:
# TODO

we check to see if the branch is at least as big as the target volume before we sample nodes from it.

In [19]:
from swc_io import get_fnames_and_abspath_from_dir, swc_to_dframe
import numpy as np
import pandas as pd

TARGET_DIM_SIZE = 25  # size in pixels for all x,y,z dims
half_size = TARGET_DIM_SIZE / 2
min_swc_dim_span = TARGET_DIM_SIZE + 5  # some leeway to make things easy
padding = min_swc_dim_span//2

BRANCHES_DIR = "../data/03_human_branches_splitted"

fnames, abs_paths = get_fnames_and_abspath_from_dir(BRANCHES_DIR)
good_swc_paths = []
good_swcs = {}
for file in abs_paths:
    nodes = swc_to_dframe(file)
    assert(isinstance(nodes, pd.DataFrame))
    
    x_span = nodes.x.max() - nodes.x.min()
    y_span = nodes.y.max() - nodes.y.min()
    z_span = nodes.z.max() - nodes.z.min()
    
    dim_size_good = np.array([x_span, y_span, z_span]) > min_swc_dim_span
    if np.all(dim_size_good):  # our SWC meets our criteria
        
        # TODO: resample SWCs here
        # use resample_swc script
        
        good_swc_paths.append(file)
        good_swcs[file] = nodes
        good_nodes = nodes.loc[(nodes.x.between(nodes.x.min()+padding,nodes.x.max()+padding)) &
                              (nodes.y.between(nodes.y.min()+padding,nodes.y.max()+padding)) &
                              (nodes.z.between(nodes.z.min()+padding,nodes.z.max()+padding))]
        if len(good_nodes) > 0:
            # grab all nodes in neighborhood around sampled node
            center_node = good_nodes.sample(n=1)
            #print(center_node.x.item())
            sample = nodes.loc[(nodes.x.between(center_node.x.item()-half_size, center_node.x.item()+half_size)) &
                              (nodes.y.between(center_node.y.item()-half_size, center_node.y.item()+half_size)) &
                              (nodes.z.between(center_node.z.item()-half_size, center_node.z.item()+half_size))]
        #print(nodes.size, good_nodes.size)
        # TODO : if no good nodes, pass
    else:
        #print("dims too small ", x_span, y_span, z_span) 
        pass
    
    

Next, we want to only randomly sample from nodes sufficiently far away from the boundary

In [15]:
# sample from the SWC
# first, we won't pick nodes too close to the edges
#filtered_nodes = np.where()
#print("dim sizes are good")

for swc_file in good_swc_paths[:1]: # TODO remove slice
    nodes = good_swcs[swc_file]
    good_nodes = 